In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torchsummary import summary

In [2]:
path = './Data/trainset/trainset/Cat/'
list_images = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [69]:
transform = transforms.Compose([transforms.ToTensor()])
traindata = torchvision.datasets.ImageFolder('./Data/trainset/trainset/', transform=transform)
testset =torchvision.datasets.ImageFolder('./Data/testset/', transform=transform)


size = [int(len(traindata)*0.9), len(traindata) - int(len(traindata)*0.9)]

trainset, validset = torch.utils.data.random_split(dataset=traindata, lengths=size)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=32,
                                          shuffle=True)

validloader = torch.utils.data.DataLoader(validset,
                                          batch_size=32,
                                          shuffle=True)

testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=32,
                                        shuffle=True)
classes = ('Cat', 'Dog')

In [70]:
cuda_available = torch.cuda.is_available()
print (cuda_available)

True


In [71]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
             # Layer 7
            nn.Conv2d(in_channels=256, out_channels=256,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 8
            nn.Conv2d(in_channels=256, out_channels=512,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # Layer 9
            nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
        )
        self.net = nn.Linear(512, 2)
        
    def forward(self, x):
        return self.net(self.conv(x).squeeze())

In [72]:
net = NNet()
learning_rate = 1e-4
if cuda_available:
    net.cuda()
# optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters())
criterion = nn.CrossEntropyLoss()

In [73]:
summary(net, (3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             448
       BatchNorm2d-2           [-1, 16, 64, 64]              32
              ReLU-3           [-1, 16, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]           4,640
       BatchNorm2d-5           [-1, 32, 64, 64]              64
              ReLU-6           [-1, 32, 64, 64]               0
         MaxPool2d-7           [-1, 32, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          18,496
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 128, 32, 32]          73,856
      BatchNorm2d-12          [-1, 128, 32, 32]             256
             ReLU-13          [-1, 128, 32, 32]               0
        MaxPool2d-14          [-1, 128,

In [74]:
def learning_rate_decay(epoch, loss, lr):
    mean_loss = np.mean(loss[epoch-10:epoch-1])
    if np.float(loss[epoch]) <= np.float(mean_loss):
        lr *= 0.1
    return lr

In [ ]:
c = 0
acc_val = []
for epoch in range(50):
    losses = []
    # Train
    for batch_idx, (images, labels) in enumerate(trainloader):
        if cuda_available:
            images, labels = images.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())

        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    net.eval()
    total = 0
    correct = 0
    for batch_idx, (images, labels) in enumerate(validloader):
        if cuda_available:
            images, targets = images.cuda(), labels.cuda()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        accuracy = 100.*(correct/total)
        acc_val.append(accuracy)

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print (learning_rate)
    print('--------------------------------------------------------------')
    net.train()
    c += 1
    if (c%10 == 0):
        learning_rate = learning_rate_decay(epoch, acc_val, learning_rate)
#     if (c%10 == 0)  and (acc_val[epoch] <= min(acc_val[epoch-4:epoch-1])):
#         learning_rate *= 0.1

Epoch : 0 Loss : 0.682 
Epoch : 0 Loss : 0.710 
Epoch : 0 Loss : 0.673 
Epoch : 0 Loss : 0.658 
Epoch : 0 Loss : 0.646 
Epoch : 0 Loss : 0.635 
Epoch : 0 Loss : 0.622 
Epoch : 0 Loss : 0.611 
Epoch : 0 Loss : 0.597 
Epoch : 0 Loss : 0.586 
Epoch : 0 Loss : 0.575 
Epoch : 0 Loss : 0.565 
Epoch : 0 Test Acc : 81.000
0.0001
--------------------------------------------------------------
Epoch : 1 Loss : 0.440 
Epoch : 1 Loss : 0.443 
Epoch : 1 Loss : 0.434 
Epoch : 1 Loss : 0.429 
Epoch : 1 Loss : 0.417 
Epoch : 1 Loss : 0.415 
Epoch : 1 Loss : 0.409 
Epoch : 1 Loss : 0.400 
Epoch : 1 Loss : 0.396 
Epoch : 1 Loss : 0.394 
Epoch : 1 Loss : 0.392 
Epoch : 1 Loss : 0.388 
Epoch : 1 Test Acc : 82.000
0.0001
--------------------------------------------------------------
Epoch : 2 Loss : 0.452 
Epoch : 2 Loss : 0.318 
Epoch : 2 Loss : 0.315 
Epoch : 2 Loss : 0.311 
Epoch : 2 Loss : 0.308 
Epoch : 2 Loss : 0.310 
Epoch : 2 Loss : 0.314 
Epoch : 2 Loss : 0.313 
Epoch : 2 Loss : 0.311 
Epoch : 2 Lo

/home/laya/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/laya/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch : 10 Loss : 0.042 
Epoch : 10 Loss : 0.039 
Epoch : 10 Loss : 0.041 
Epoch : 10 Loss : 0.046 
Epoch : 10 Loss : 0.043 
Epoch : 10 Loss : 0.040 
Epoch : 10 Loss : 0.043 
Epoch : 10 Loss : 0.043 
Epoch : 10 Loss : 0.043 
Epoch : 10 Loss : 0.046 
Epoch : 10 Loss : 0.047 
Epoch : 10 Test Acc : 91.000
0.0001
--------------------------------------------------------------
Epoch : 11 Loss : 0.009 
Epoch : 11 Loss : 0.030 
Epoch : 11 Loss : 0.028 
Epoch : 11 Loss : 0.033 
Epoch : 11 Loss : 0.032 
Epoch : 11 Loss : 0.034 
Epoch : 11 Loss : 0.034 
Epoch : 11 Loss : 0.037 
Epoch : 11 Loss : 0.037 
Epoch : 11 Loss : 0.036 
Epoch : 11 Loss : 0.036 
Epoch : 11 Loss : 0.035 
Epoch : 11 Test Acc : 87.000
0.0001
--------------------------------------------------------------
Epoch : 12 Loss : 0.056 
Epoch : 12 Loss : 0.042 
Epoch : 12 Loss : 0.037 
Epoch : 12 Loss : 0.035 
Epoch : 12 Loss : 0.035 
Epoch : 12 Loss : 0.036 
Epoch : 12 Loss : 0.040 
Epoch : 12 Loss : 0.036 
Epoch : 12 Loss : 0.035 
Ep

Epoch : 30 Loss : 0.012 
Epoch : 30 Loss : 0.012 
Epoch : 30 Test Acc : 89.000
1.0000000000000002e-06
--------------------------------------------------------------
Epoch : 31 Loss : 0.000 
Epoch : 31 Loss : 0.010 
Epoch : 31 Loss : 0.007 
Epoch : 31 Loss : 0.006 
Epoch : 31 Loss : 0.008 
Epoch : 31 Loss : 0.008 
Epoch : 31 Loss : 0.008 
Epoch : 31 Loss : 0.009 
Epoch : 31 Loss : 0.010 
Epoch : 31 Loss : 0.010 
Epoch : 31 Loss : 0.009 
Epoch : 31 Loss : 0.009 
Epoch : 31 Test Acc : 90.000
1.0000000000000002e-06
--------------------------------------------------------------
Epoch : 32 Loss : 0.000 
Epoch : 32 Loss : 0.011 
Epoch : 32 Loss : 0.017 
Epoch : 32 Loss : 0.013 
Epoch : 32 Loss : 0.011 
Epoch : 32 Loss : 0.010 
Epoch : 32 Loss : 0.011 
Epoch : 32 Loss : 0.009 
Epoch : 32 Loss : 0.009 
Epoch : 32 Loss : 0.009 
Epoch : 32 Loss : 0.008 
Epoch : 32 Loss : 0.008 
Epoch : 32 Test Acc : 90.000
1.0000000000000002e-06
--------------------------------------------------------------
Epoch

In [45]:
torch.save(net.state_dict(), './model/adam.pwf')

In [47]:
# torch.save(net.state_dict(), 
#            os.path.join('saved_model', 'cnnT1_epoch_{}_iter_{}_loss_{}_acc_{}_{}.t7'.format(epoch + 1, i,
#            last_loss, acc, datetime.datetime.now().strftime("%b_%d_%H:%M:%S"))))

In [55]:
# np_image = trainset[0
# img = torch.from_numpy(np_image)
# img = img.view(img.shape[1], img.shape[2], img.shape[0])
# # img = img / 2+0.5
# # npimg = img.numpy()
# # plt.imshow(np.transpose(npimg, (1, 2, 0)))
# plt.imshow(img)
# plt.show()